In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple

In [32]:
# Train paths
kmnist_train_images = "./kmnist-train-imgs.npz"
kmnist_train_labels = "./kmnist-train-labels.npz"

# Test paths
kmnist_test_images = "./kmnist-test-imgs.npz"
kmnist_test_labels = "./kmnist-test-labels.npz"

In [33]:
# KMNIST DataLoader Class

class DataLoader:
    """
    A DataLoader class that loads KMNIST data from .npz files for images and labels.
    Each .npz contains "arr_0".
    """
    def __init__(self, train_images_path: str, train_labels_path: str, test_images_path: str, test_labels_path: str):
        self.train_images_path = train_images_path
        self.train_labels_path = train_labels_path
        self.test_images_path = test_images_path
        self.test_labels_path = test_labels_path

    def load_data(self) -> Tuple[np.array]:
        """
        Loads arrays from the given .npz files.
        
        Returns:
            (Tuple[np.array]): Returns a tuple of numpy arrays.
        """
        X_train = np.load(self.train_images_path)["arr_0"]
        y_train = np.load(self.train_labels_path)["arr_0"]

        X_test = np.load(self.test_images_path)["arr_0"]
        y_test = np.load(self.test_labels_path)["arr_0"]

        return (X_train, y_train), (X_test, y_test)


In [37]:
# Loading the data 
loader = DataLoader(kmnist_train_images, kmnist_train_labels, kmnist_test_images, kmnist_test_labels)

(X_train, y_train), (X_test, y_test) = loader.load_data()